# NOAA PMEL CO2 Mooring data example
Created by Sophie Chu of NOAA PMEL and modified by Nancy Williams of USF CMS for use with Ocean Biogeochemical Dynamics Lab course
- Downloading data via url
- Creating a dataframe using pandas
- Calculating monthly means and standard deviations
- Plotting using matplotlib
- Decomposing pCO2 seasonality into thermal and nonthermal components

In [ ]:
# import libraries needed for this analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
output_dir='generated/' # define where you want your figures to go

In [ ]:
# Make a function for data download via url

def download_url(url):
  print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  file_name_start_pos = url.rfind("/") + 1
  file_name = url[file_name_start_pos:]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    with open(file_name, 'wb') as f:
      for data in r:
        f.write(data)

In [ ]:
# Use above function to download data via url and save file
# Let's start with the KEO mooring off of Hawaii.
download_url("https://www.pmel.noaa.gov/co2/timeseries/KEO.txt")

In [ ]:
# read text file into dataframe
keo = pd.read_table('KEO.txt', delim_whitespace=True, comment ='#', header=0,
                       names=('date','time', 'SST', 'SSS', 'pCO2_sw', 'pCO2_air', 'xCO2_air', 'pH_sw'))

In [ ]:
# take a quick look at the beginning and end of the dataframe
keo.head

In [ ]:
# function to define data time as index and add depth and month columns

def surfdf (_df, date, time, dpth):
    _df['datetime64_ns'] = pd.to_datetime(_df.date+ ' ' + _df.time) # combine date and time into one column
    _df.index = _df.datetime64_ns # set datetime column as index
    _df['Depth'] = dpth # add depth of surface measurements as column
    _df['month'] = _df['datetime64_ns'].dt.month.values #make month column
    
    return _df

# apply function to dataframe

df_keo = surfdf(keo, keo.date, keo.time, 0.5)

In [ ]:
# take a quick look at the beginning and end of the new dataframe
df_keo.head

In [ ]:
# take a quick look at the different data types of variables in the data frame
df_keo.dtypes

In [ ]:
# quick plot
df_keo.pCO2_sw.plot()

In [ ]:
# plotting SST and SSS on one plot
fig, ax = plt.subplots()
im = ax.scatter(df_keo.index, df_keo.SST, s=1)
im2 = ax.scatter(df_keo.index,df_keo.SSS, s=1)
ax.set_ylabel('SST and SSS') # fontsize=fs)
plt.title('KEO Mooring')
plt.savefig(output_dir+'keo_ts.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.show() # displays plot

In [ ]:
# get average monthly values for dataset
df_keo_avgmonth = df_keo.groupby('month').agg(['mean', 'std', 'count'])
df_keo_avgmonth # displays the results

In [ ]:
df_keo_avgmonth.info()

#### *Note: Ideally, we would detrend this pCO2 data first, because we know that atospheric pCO2 increases each year (https://www.esrl.noaa.gov/gmd/ccgg/trends/) and that the ocean generally increases at a similar pace. For this analysis, it will not be important but it is best practice to detrend your data before calculating monthly climatological seasonal cycles.

In [ ]:
# find max monthly mean
df_keo_avgmonth.pCO2_sw['mean'].max()

In [ ]:
# plot monthly average SST
fig, ax = plt.subplots()
im = ax.errorbar(df_keo_avgmonth.index, df_keo_avgmonth.SST['mean'], 
                df_keo_avgmonth.SST['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('SST')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.title('KEO Mooring')
plt.savefig(output_dir+'keo_avgmonth_sst.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

Next, we want to calculate solubility-related changes in pCO2,'thermal component' and decompose from the non-thermal component. First, let's look at the monthly pCO2 data.

In [ ]:
# plot monthly averages for pCO2
fig, ax = plt.subplots()
im = ax.errorbar(df_keo_avgmonth.index, df_keo_avgmonth.pCO2_sw['mean'], 
                df_keo_avgmonth.pCO2_sw['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('seawater pCO2')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.title('KEO Mooring')
plt.savefig(output_dir+'keo_avgmonth_pCO2sw.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

In [ ]:
# Decomposing the thermal and nonthermal components of pCO2
SSTi=df_keo.SST[0]
pCO2_swi=df_keo.pCO2_sw[0]
pCO2_thermal=pCO2_swi+pCO2_swi*.0423*(df_keo.SST-SSTi) # 0.0423 as per Takahashi et al. (1993)

In [ ]:
# plotting pCO2 and pCO2thermal on one plot
fig, ax = plt.subplots()
im = ax.scatter(df_keo.index, df_keo.pCO2_sw,label='Observed pCO2', s=1)
im3 = ax.scatter(df_keo.index, df_keo.pCO2_sw-pCO2_thermal+pCO2_swi,label='nonthermal', s=1)
im2 = ax.scatter(df_keo.index, pCO2_thermal,label='thermal', s=1)
ax.set_ylabel('pCO2(microatm.)') # fontsize=fs)
ax.legend()
plt.title('KEO Mooring')
plt.savefig(output_dir+'keo_pCO2thermal.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.show() # displays plot

### Is pCO2 at KEO driven more by thermal or nonthermal (biophysical) effects?

Now, let's compare to another mooring. You can go to https://www.pmel.noaa.gov/co2/story/CO2+Data+Discovery and find another mooring. Try to find one where the biophysical effects dominate. Where do you think you might find such a mooring?

In [ ]:
# Dynamically define the new mooring name
MooringName='PAPA'
download_url("https://www.pmel.noaa.gov/co2/timeseries/"+MooringName+".txt")

In [ ]:
# read text file into dataframe
Mooring = pd.read_table(MooringName+'.txt', delim_whitespace=True, comment ='#', header=0,
                       names=('date','time', 'SST', 'SSS', 'pCO2_sw', 'pCO2_air', 'xCO2_air', 'pH_sw'))

In [ ]:
# apply function to dataframe (this function was defined earlier in the code)
df_Mooring = surfdf(Mooring, Mooring.date, Mooring.time, 0.5)
# quick plot
df_Mooring.pCO2_sw.plot()

In [ ]:
# plotting SST and SSS on one plot
fig, ax = plt.subplots()
im = ax.scatter(df_Mooring.index, df_Mooring.SST, s=1)
im2 = ax.scatter(df_Mooring.index,df_Mooring.SSS, s=1)
ax.set_ylabel('SST and SSS') # fontsize=fs)
plt.savefig(output_dir+MooringName+'_ts.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.title(MooringName+' Mooring')
plt.show() # displays plot

In [ ]:
# get average monthly values for dataset
df_Mooring_avgmonth = df_Mooring.groupby('month').agg(['mean', 'std', 'count'])
# find max monthly mean
df_Mooring_avgmonth.pCO2_sw['mean'].max()

In [ ]:
# plot monthly average SST
fig, ax = plt.subplots()
im = ax.errorbar(df_Mooring_avgmonth.index, df_Mooring_avgmonth.SST['mean'], 
                df_Mooring_avgmonth.SST['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('SST')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.title(MooringName+' Mooring')
plt.savefig(output_dir+MooringName+'_avgmonth_sst.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

In [ ]:
# plot monthly averages for pCO2
fig, ax = plt.subplots()
im = ax.errorbar(df_Mooring_avgmonth.index, df_Mooring_avgmonth.pCO2_sw['mean'], 
                df_Mooring_avgmonth.pCO2_sw['std'])#, marker= 's', color='C0',elinewidth=1, linewidth=3)
ax.set_xlabel('Month')#, fontsize=fs)
ax.set_ylabel('seawater pCO2')#, fontsize=fs)
ax.set_xticks(np.arange(1, 13, step=2))
plt.title(MooringName+' Mooring')
plt.savefig(output_dir+MooringName+'_avgmonth_pCO2sw.png', bbox_inches='tight', dpi=600)
plt.show() # displays plot

In [ ]:
# Decomposing the thermal and nonthermal components of pCO2
SSTi=df_Mooring.SST[0]
pCO2_swi=df_Mooring.pCO2_sw[0]
pCO2_thermal=pCO2_swi+pCO2_swi*.0423*(df_Mooring.SST-SSTi) # 0.0423 as per Takahashi et al. (1993)

# plotting pCO2 and pCO2thermal on one plot
fig, ax = plt.subplots()
im = ax.scatter(df_Mooring.index, df_Mooring.pCO2_sw,label='Observed pCO2', s=1)
im3 = ax.scatter(df_Mooring.index, df_Mooring.pCO2_sw-pCO2_thermal+pCO2_swi,label='nonthermal', s=1)
im2 = ax.scatter(df_Mooring.index, pCO2_thermal,label='thermal', s=1)
ax.set_ylabel('pCO2(microatm.)') # fontsize=fs)
ax.legend()
plt.title(MooringName+' Mooring')
plt.savefig(output_dir+MooringName+'_pCO2thermal.png', bbox_inches='tight', dpi=600) # saves plot as image file
plt.show() # displays plot